# Numerical integration with Python

In the text, [Euler's Method](http://roboscience.org/book/html/Simulation/MovingDifferential.html#a-numerical-approach) is used to numerically integrate the differential drive forward kinematics in order to find a path through the workspace given by some path through the configuration space. This is an exposition of numerical integration topics related to solving systems of ODEs.

First, we need to get some notation down. We need to represent the differential drive forward kinematic equations as a system of differential equations with the form

$$\frac{\mathrm d \vec y}{\mathrm d t} = f(t, \vec y)$$

Where

$$\vec y = \begin{bmatrix}
    y_1(t) \\
    y_2(t) \\
    \vdots \\
    y_n(t)
\end{bmatrix}$$

is the vector of state variables, and

$$\frac{\mathrm d \vec y}{\mathrm d t} = \begin{bmatrix}
    \frac{\mathrm d y_1}{\mathrm d t} \\
    \frac{\mathrm d y_2}{\mathrm d t} \\
    \vdots                            \\
    \frac{\mathrm d y_n}{\mathrm d t}
\end{bmatrix}$$

is the vector of their time derivatives. So in total, we have a system of first order ODEs of the form

$$\begin{bmatrix}
    \frac{\mathrm d y_1}{\mathrm d t} \\
    \frac{\mathrm d y_2}{\mathrm d t} \\
    \vdots                            \\
    \frac{\mathrm d y_n}{\mathrm d t}
\end{bmatrix} = \begin{bmatrix}
    f_1(t, y_1, y_2, \dots, y_n) \\
    f_2(t, y_1, y_2, \dots, y_n) \\
    \vdots                       \\
    f_n(t, y_1, y_2, \dots, y_n)
\end{bmatrix}$$

Notice that the differential drive forward kinematics have precisely this form when we write $\dot \phi_1$ and $\dot \phi_2$ as functions of $t$!

$$\begin{bmatrix}
    \frac{\mathrm d x}{\mathrm d t} \\
    \frac{\mathrm d y}{\mathrm d t} \\
    \frac{\mathrm d \theta}{\mathrm d t}
\end{bmatrix} = \begin{bmatrix}
    \frac{r}{2} \left(\dot \phi_1(t) + \dot \phi_2(t)\right) \cos \theta \\
    \frac{r}{2} \left(\dot \phi_1(t) + \dot \phi_2(t)\right) \sin \theta \\
    \frac{r}{2L}\left(\dot \phi_1(t) - \dot \phi_2(t)\right)
\end{bmatrix}$$

## Eulers Method

The [book](http://roboscience.org/book/html/Simulation/MovingDifferential.html#a-numerical-approach) walks through the discretization and formulation of Euler's Method. Here's a quick summary.

We discretize $t$ by chunking it with step size $\Delta t$. This produces the discretized forward kinematics formulas

$$\begin{array}{l}
    x_{k+1} = x_k + \frac{r\Delta t}{2} \left(\omega_{1, k} + \omega_{2, k}\right)\cos(\theta_k) \\
    y_{k+1} = y_k + \frac{r\Delta t}{2} \left(\omega_{1, k} + \omega_{2, k}\right)\sin(\theta_k) \\
    \theta_{k+1} = \theta_k + \frac{r\Delta t}{2L} \left(\omega_{1, k} - \omega_{2, k}\right)
\end{array}$$

where $\omega_{i, k}$ is the $i$th wheel speed at time step $t_k$.

Now let's use Euler's Method to find the path that a robot with $r = 0.5$ and $L = 0.6$ takes when it has wheel speeds determined by

$$\dot \phi_1 = 0.25 \sin(2t)$$
$$\dot \phi_2 = 0.5 \cos(2 + t)$$

for $0 \leq t \leq 5$.

In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import seaborn as sns

sns.set()

In [ ]:
def phi1_dot(t):
    """Returns the right wheel speed at the given time."""
    return 0.25 * (t ** 1.5)

def phi2_dot(t):
    return 0.75 * t

In [ ]:
r = 0.5
L = 0.6

In [ ]:
def x_dot(w1, w2, theta):
    """The differential drive FK x velocity"""
    return (r / 2.0) * (w1 + w2) * np.cos(theta)

def y_dot(w1, w2, theta):
    """The differential drive FK y velocity"""
    return (r / 2.0) * (w1 + w2) * np.sin(theta)

def theta_dot(w1, w2):
    return (r / (2.0 * L)) * (w1 - w2)

Now that we have the pieces of the puzzle laying on the table, let's put them together.

In [ ]:
def path(x0, ta, tb, step):
    """Uses Euler's Method to find the path of the robot starting at x0
    as time goes from ta to tb with some step size.
    """
    points = []

    for t in np.arange(ta, tb+step, step):
        # Unpack the previous point into its components.
        x, y, theta = x0
        # Compute the wheel speeds at the given time.
        w1 = phi1_dot(t)
        w2 = phi2_dot(t)
        # Update the theta component first, because x, and y depend on theta.
        theta = theta + step * theta_dot(w1, w2)
        # Update the x and y components.
        x = x + step * x_dot(w1, w2, theta)
        y = y + step * y_dot(w1, w2, theta)
        # Repack the updated components into the current position estimate.
        x0 = np.array([x, y, theta])
        # Save the current position estimate.
        points.append(x0)

    # Convert the Python list of points to a numpy array
    return np.array(points)

Now consider the robot path from $t=0$ to $t=5$ starting at $(0, 0, 0)$ with a fairly large step size to exaggerate one of the shortcomings of Euler's method.

In [ ]:
euler_path = path((0, 0, 0), 0, 5, step=0.5)

# Get the x and y components into their own arrays so we can plot them
euler_x = [p[0] for p in euler_path]
euler_y = [p[1] for p in euler_path]
euler_theta = [p[2] for p in euler_path]

In [ ]:
plt.title('Robot path estimated with Euler\'s Method')
plt.plot(euler_x, euler_y)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

## A Better Way

Python is a language with a *rich* ecosystem of libraries. In particular, Python has an excellent scientific computation libraries that do everything from advanced mathematics to machine learning, to data analytics, to simulating quantum computers. A good rule of thumb is that if you have a need to perform some general task, there's a Python library that implements at least some form of that task better than you could, should you attempt to.

Thus, we should look at how to perform numerical integration the *right way* in Python. After a quick Google search, you should find [`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) or [`scipy.integrate.odeint`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html). Here, we'll use `solve_ivp`.

Looking at the excellent SciPy [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) we see that `solve_ivp` has the call signature

```python
solve_ivp(fun, t_span, y0, t_eval=None)
```

where `f(t, y)` is a Python function defining the right hand side of the system of ODEs
$$\frac{\mathrm d \vec y}{\mathrm d t} = f(t, \vec y)$$
integrated over the `t_span`
$$a \leq t \leq b$$
and `y0` is the initial condition
$$y_0 = y(a)$$

If we wish to plot the path of the robot, we also need to pass in the optional `t_eval` parameter to tell it at which times to save the $(x, y, \theta)$ points along the path.

So we define the function as follows:

In [ ]:
def f(t, vars):
    """The differential drive robot FK."""
    # Unpack the current step estimates of the values of the state variables
    x, y, theta = vars
    # Use the functions we defined earlier
    w1 = phi1_dot(t)
    w2 = phi2_dot(t)
    # Return a vector of the time derivatives
    return x_dot(w1, w2, theta), y_dot(w1, w2, theta), theta_dot(w1, w2)

Now that we have our function defined, we need to integrate it. The default solution method `solve_ivp` uses is 'RK45' (Explicit Runge-Kutta method of order 5(4)). For most cases, this is sufficient, but there are other options should you need them.

In [ ]:
from scipy.integrate import solve_ivp
result = solve_ivp(f, (0, 5), (0, 0, 0), t_eval=np.linspace(0, 5, 100))
# The result that solve_ivp returns is a complex object, with lots of fields.
# Refer to the documentation for details.
rk4_x, rk4_y, rk4_theta = result.y

Now we plot the results.

In [ ]:
plt.title('Robot path estimated with RK4')
plt.plot(rk4_x, rk4_y)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Now we should compare the results with what we got using Euler's Method

In [ ]:
plt.title('Robot path comparison')
plt.plot(rk4_x, rk4_y, label='RK4')
plt.plot(euler_x, euler_y, label='Euler')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Note that Euler's method over estimates the curve in the arc. This an inherent problem with Euler's Method that RK4 was invented to solve. Now, of course, we used a giant step size to exaggerate this problem. Let's try again with a more reasonable step size.

In [ ]:
euler_path = path((0, 0, 0), 0, 5, step=0.1)

# Get the x and y components into their own arrays so we can plot them
euler_x = [p[0] for p in euler_path]
euler_y = [p[1] for p in euler_path]
euler_theta = [p[2] for p in euler_path]

In [ ]:
plt.title('Robot path comparison')
plt.plot(rk4_x, rk4_y, label='RK4')
plt.plot(euler_x, euler_y, label='Euler')
plt.legend()
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

Notice that this is much closer, but still off. Be aware of this problem when using Euler's Method. It's computationally simpler, but over time, error can accumulate, especially if you have not tuned your step size to your problem.